In [1]:
import habitat

/home/green-tea/miniconda3/envs/habitat/lib/python3.6/site-packages/numba-0.46.0-py3.6-linux-x86_64.egg/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import uuid
import os
from typing import Any
import quaternion
import random

In [3]:
class AgentPosition:
    angle_weight = 12
    coordinate_weight = 4
    min_diff = 2
    
    def __init__(self, position, rotation):
        self.position = position
        self.rotation = rotation
        
    def get_position(self):
        return self.position
    
    def get_rotation(self):
        return self.rotation
        
    def radian_to_grade(self, angle):
        return angle / 2 / math.pi * 360 + 180
        
    def print(self):
        print('Position= ', self.position, sep='')
        print('Rotation=', self.rotation, sep='')
        
    
    @staticmethod
    def get_difference_position(p1, p2):
        return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2 + (p1[2] - p2[2])**2)
    
    @staticmethod
    def get_difference_rotation(r1, r2):
        f1 = quaternion.as_float_array(r1)[0]
        f2 = quaternion.as_float_array(r1)[2]
        
        s1 = quaternion.as_float_array(r2)[0]
        s2 = quaternion.as_float_array(r2)[2]
        return math.sqrt((f1 - s1) ** 2 + (f2 - s2) ** 2)
    
    @staticmethod
    def get_diff(p1, p2):
        position1 = p1.get_position()
        position2 = p2.get_position()
        position_diff = AgentPosition.get_difference_position(position1, position2)
        
        rotation1 = p1.get_rotation()
        rotation2 = p2.get_rotation()
        
        rotation_diff = AgentPosition.get_difference_rotation(rotation1, rotation2)
        
        return position_diff * AgentPosition.coordinate_weight + rotation_diff * AgentPosition.angle_weight
        
    @staticmethod
    def is_close(p1, p2):
        
        diff = AgentPosition.get_diff(p1, p2)
        
#         print(diff)
        
        return AgentPosition.min_diff > diff
    
    def create_np_array(self):
        return np.array([
            self.position[0] * AgentPosition.coordinate_weight,
            self.position[1] * AgentPosition.coordinate_weight,
            self.position[2] * AgentPosition.coordinate_weight,
            quaternion.as_float_array(self.rotation)[0] * AgentPosition.angle_weight,
            quaternion.as_float_array(self.rotation)[2] * AgentPosition.angle_weight
        ])
        
        


In [4]:

# Define the sensor and register it with habitat
# For the sensor, we will register it with a custom name
@habitat.registry.register_sensor(name="my_supercool_sensor")
class AgentPositionSensor(habitat.Sensor):
    def __init__(self, sim, config, **kwargs: Any):
        super().__init__(config=config)

        self._sim = sim
        # Prints out the answer to life on init
        print("The answer to life is", self.config.ANSWER_TO_LIFE)

    # Defines the name of the sensor in the sensor suite dictionary
    def _get_uuid(self, *args: Any, **kwargs: Any):
        return "agent_position"

    # Defines the type of the sensor
    def _get_sensor_type(self, *args: Any, **kwargs: Any):
        return habitat.SensorTypes.POSITION

    # Defines the size and range of the observations of the sensor
    def _get_observation_space(self, *args: Any, **kwargs: Any):
        return 5

    # This is called whenver reset is called or an action is taken
    def get_observation(
        self, observations, *args: Any, episode, **kwargs: Any
    ):
        return AgentPosition(self._sim.get_agent_state().position, self._sim.get_agent_state().rotation).create_np_array()

In [5]:
config = habitat.get_config("/home/green-tea/all_projects/habitat/segmentation/configs/challenge_objectnav2020.local.rgbd.yaml")

In [6]:
config.defrost()

config.DATASET.DATA_PATH = "/home/green-tea/all_stash/matterport3D/data/val_mini.json.gz"
config.DATASET.SCENES_DIR = "/home/green-tea/all_stash/matterport3D/data4/v1/tasks"

config.SIMULATOR.AGENT_0.SENSORS.append('SEMANTIC_SENSOR')

config.TASK.AGENT_POSITION_SENSOR = habitat.Config()
config.TASK.AGENT_POSITION_SENSOR.TYPE = "my_supercool_sensor"
config.TASK.AGENT_POSITION_SENSOR.ANSWER_TO_LIFE = 42
config.TASK.SENSORS.append("AGENT_POSITION_SENSOR")

config.freeze()

In [7]:
env = habitat.Env(config)

2020-04-09 11:11:53,022 Initializing dataset ObjectNav-v1
2020-04-09 11:11:53,067 initializing sim Sim-v0
I0409 11:11:58.318681 28314 simulator.py:143] Loaded navmesh /home/green-tea/all_stash/matterport3D/data4/v1/tasks/mp3d/x8F5xyUWy9e/x8F5xyUWy9e.navmesh
I0409 11:11:58.319488 28314 simulator.py:155] Recomputing navmesh for agent's height 0.88 and radius 0.18.
2020-04-09 11:11:58,413 Initializing task ObjectNav-v1


The answer to life is 42


In [8]:
DATASET_DIRECTORY_X = '/home/green-tea/all_projects/habitat/mask-r-cnn/X'
DATASET_DIRECTORY_Y = '/home/green-tea/all_projects/habitat/mask-r-cnn/Y'

In [9]:
def prepare_semantic_observation(observations):
    scene = env.sim.semantic_annotations()
    instance_id_to_label_id = {int(obj.id.split("_")[-1]): obj.category.index() for obj in scene.objects}
    mapping = np.array([ instance_id_to_label_id[i] for i in range(len(instance_id_to_label_id)) ])

    return np.take(mapping, observations['semantic'])

In [10]:
scene = env.sim.semantic_annotations()
index_to_title_map = {obj.category.index(): obj.category.name() for obj in scene.objects }
index_to_title_map[-1] = 'nope'
print(index_to_title_map)

{1: 'wall', 40: 'misc', 16: 'stairs', 4: 'door', 2: 'floor', 39: 'objects', 17: 'ceiling', 0: 'void', 14: 'plant', 24: 'column', -1: 'nope', 5: 'table', 21: 'mirror', 15: 'sink', 7: 'cabinet', 28: 'lighting', 34: 'seating', 20: 'towel', 12: 'curtain', 9: 'window', 23: 'shower', 38: 'clothes', 11: 'bed', 35: 'board_panel', 37: 'appliances', 31: 'shelving', 26: 'counter', 22: 'tv_monitor', 3: 'chair'}


In [11]:
def save_image_for_dataset(observations):
    rgb = observations['rgb']
    
    image = Image.fromarray(rgb)
    u = uuid.uuid1()
    
    image.save(DATASET_DIRECTORY_X + "/" + str(u) + '.png')
    
#     semantic = prepare_semantic_observation(observations)
    semantic = observations['semantic']
    
    unique_values = np.unique(semantic)
    objects = []
    
    os.mkdir(DATASET_DIRECTORY_Y + "/" + str(u))
    
    for value in unique_values:
        obj = (semantic == value)
        
        title = str(value)
        
        image = Image.fromarray(obj)
        image.save(DATASET_DIRECTORY_Y + "/" + str(u) + "/" + title + ".png")

In [17]:
observations = env.reset()

In [18]:
print(prepare_semantic_observation(observations))

[[17 17 17 ...  1  1  1]
 [17 17 17 ...  1  1  1]
 [31 31 31 ...  1  1  1]
 ...
 [ 2  2  2 ...  1  1  1]
 [ 2  2  2 ...  1  1  1]
 [ 2  2  2 ...  1  1  1]]


In [19]:
save_image_for_dataset(observations)

In [12]:
def check_action(cur_action, last_action, is_bump):
    if is_bump and cur_action == 'MOVE_FORWARD':
        return False
    
    if cur_action == 'TURN_LEFT' and last_action == 'TURN_RIGHT':
        return False
    if cur_action == 'TURN_RIGHT' and last_action == 'TURN_LEFT':
        return False

    return True

In [13]:
def get_random_action(last, is_bump):
    action = random.choice(list(action_mapping))

    while not(check_action(action, last, is_bump)):
        action = random.choice(list(action_mapping))

    return action

In [14]:
class Saver:
    def __init__(self):
        self.count = 0
        self.path = 'images'
        self.all_positions = {}
        
    def save_image(self, observations, env):
        cur_scene = env.current_episode.scene_id
        
        if self.check_exist_image(observations, cur_scene):
            return;

        save_image_for_dataset(observations)

        self.count += 1
        
        if cur_scene in self.all_positions:
            self.all_positions[cur_scene] = np.append(self.all_positions[cur_scene], [observations['agent_position']], axis=0)
        else:
            self.all_positions[cur_scene] = np.array([observations['agent_position']])
        
    def check_exist_image(self, observations, cur_scene):
        cur_position = observations['agent_position']
        
        if not(cur_scene in self.all_positions):
            return False
        
        min_diff = np.sqrt(np.square(self.all_positions[cur_scene] - cur_position).sum(axis=1).min())
        
        return min_diff <= 2

In [15]:
def check_bump(last_observations, observations):
    return np.sqrt(np.square(last_observations['agent_position'] - observations['agent_position']).sum()) <= 0.5

In [16]:
action_mapping = {
    1: 'MOVE_FORWARD',
    2: 'TURN_LEFT',
    3: 'TURN_RIGHT'
}

In [17]:
saver = Saver()

index = 0
for i in range(len(env.episodes)):
    observations = env.reset()

    last = ''
    is_bump = False

    for i in range(300):
        action = get_random_action(last, is_bump)
        last = action

        last_observations = observations
        observations = env.step(action)

        is_bump = check_bump(last_observations, observations)
        
        if index == 1000:
            break
            
        index += 1

        saver.save_image(observations, env)
        
    if index == 1000:
        break

KeyboardInterrupt: 

In [56]:
kek = observations['rgb'].reshape([320, 240, -1])

In [60]:
kek.shape
img = Image.fromarray(kek)

TypeError: Cannot handle this data type: (1, 1, 12), |u1